<a href="https://colab.research.google.com/github/rajshaiwalla/NLP-Submissions/blob/master/J043_NLP_M2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading & Preprocessing

In [1]:
! wget https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true #reading from github
! mkdir newsarticles
! unzip 10k-german-news-articles.zip?raw=true -d newsarticles

--2020-03-03 18:23:25--  https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 18:23:25--  https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 18:23:25--  https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.13

In [55]:
import numpy as np
import pandas as pd
import os
import re
import nltk
import warnings
import random
import pickle
from nltk import word_tokenize
from gensim import models, corpora
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.metrics.pairwise import cosine_similarity
import string
from string import digits
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import lightgbm as lgb
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim
from pyLDAvis import sklearn as sklearn_lda
import gensim
from gensim import corpora
import pickle

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
german_stop_words = stopwords.words('german')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [0]:
posts_df=pd.read_csv("/content/newsarticles/Posts.csv") #reading csv and making it in a dataframe
articles_df=pd.read_csv("/content/newsarticles/Articles.csv")
df=pd.read_csv("/content/newsarticles/Articles.csv")

In [0]:
dfm = pd.merge(posts_df, articles_df, on="ID_Article") #merging df
dft = pd.merge(posts_df, articles_df, on="ID_Article")

In [5]:
df.head()

,ID_Article,Path,publishingDate,Title,Body
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ..."
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro..."
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro..."


In [6]:
df.shape

(12087, 5)

In [7]:
df_2k = df.iloc[:2000] #selecting only 2000 rows
df_2kt = dft.iloc[:2000]
df_2k.shape

(2000, 5)

In [0]:
def punctuation(text): #funtion to remove punctuation
  
    doc = text.lower()
    doc = re.sub(r'<.*?>', ' ', doc)
    doc = re.sub(r'\n', '', doc)
    doc = re.sub(r'\t', '', doc)
    doc = re.sub(r'\r', '', doc)
    doc = re.sub(r'[^A-Za-z\s]', ' ', doc)
    return doc

def short_words(text): #funtion to remove words whose length is less than three
    s_words = ' '.join([w for w in text.split() if len(w)>3])
    return s_words

def stop_words(text): #funtion to remove stop words
    split = text.split()
    tokens = [item for item in split if item not in german_stop_words]
    final = ' '.join([w for w in tokens])
    return final

In [0]:
def preprocessing(text): #function combined of punctuation, short_words, stop_words
    ctext = punctuation(text)
    ctext = short_words(ctext)
    ctext = stop_words(ctext)
    return ctext

In [0]:
def tokenize(string): #function to tokenize
    return nltk.tokenize.WordPunctTokenizer().tokenize(string)

# Body

In [11]:
df_2k['Body']=df_2k['Body'].apply(preprocessing) #applying preprocessing to Body Column

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
print(len(df_2k['Body'][0].split()))

173


In [13]:
print(len(df_2k['Body'][0]))

1505


In [14]:
df_2k['Body'][0]

'abonnieren unserer newsletter lassen aktuelle nachrichten mail zuschicken derstandard bietet vielzahl themenbezogener newsletter motto nachrichten echtzeit liefert derstandard wichtigsten meldungen tages mail html textversion derstandard newsletter nnen kostenloses newsletter abonnement bequem abmelden newsletter berblick mail newsroom spitzenmeldungen newsroom ressorts mail meinung aktuelle kommentare kolumnen mail internationalpolitik news internationale geschehen mail innenpolitik news innenpolitische geschehen mail wirtschaft nachrichten unternehmens wirtschaftsbranche mail finanz finanz news mail webstandard neueste netz technologie mail sport spitzenmeldungen sportressort mail panorama meldungen chronik gesellschaft umwelt wien mail etat aktuelles medien werbung sowie newsflash mail kultur aktuelle kultur nachrichten alert mail brandaktuelle meldungen besonderer wichtigkeit liefert mail chentliche newsletter weekly mail mail informiert jeweils wochenmitte neuigkeiten rund angebo

In [15]:
print(len(df_2kt['Body_y'][0]))

2924


In [16]:
tfidfb = TfidfVectorizer() #defining tfidf vectorizer for body
c_corpus = ['{}'.format(text) for text in df_2k['Body']] #forming corpus
c_corpus
XB = tfidfb.fit_transform(c_corpus) #fitting tfidf for body
f_names = tfidfb.get_feature_names()
dense = XB.todense().tolist()
new_df = pd.DataFrame(dense,columns=f_names)
new_df.head()

,aaaaaaatemlos,aachen,aalen,aalglatte,aaron,aatt,abac,abadi,abarbeiten,abba,abbas,abbau,abbauarbeiten,abbaubank,abbaubaren,abbaueinheit,abbauen,abbaugesellschaft,abbaut,abbauteil,abbei,abberuft,abbey,abbezahlt,abbiegen,abbild,abbilde,abbilden,abbildung,abbildungen,abblitzen,abbr,abbrach,abbrechen,abbremsblase,abbremsen,abbremst,abbringen,abbruch,abbruchh,...,zwischenl,zwischenlanden,zwischenmahlzeiten,zwischenmenschliche,zwischenmoderationen,zwischennutzungen,zwischennutzungsprojekt,zwischenruf,zwischenrufe,zwischenrufen,zwischenrufs,zwischenschritt,zwischenspiele,zwischenstadium,zwischenstand,zwischenstation,zwischenstopp,zwischent,zwischenzeit,zwischenzeitlich,zwischenzeitlichem,zwischenzeitlichen,zwist,zwistigkeiten,zwitschern,zwittern,zyklischen,zyklus,zylinder,zylinderkopf,zynisch,zynischer,zynismus,zypern,zyperngras,zyperns,zyprioten,zypriotischen,zytostatika,zywteyx
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.104473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
warnings.simplefilter("ignore", DeprecationWarning)
 
# Helper function
def print_topics(model, tfidf, n_top_words):
    words = tfidf.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below
number_topics = 5
number_words = 5
# Create and fit the LDA model
ldab = LDA(n_components=number_topics, n_jobs=-1)
ldab.fit(XB)
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(ldab, tfidfb, number_words)

Topics found via LDA:

Topic #0:
volltext basierenden agenturmeldungen artikels rechtlichen

Topic #1:
android nexus nintendo klimt ampelp

Topic #2:
sporttabellenteaser params ajax success ball

Topic #3:
prozent euro mehr wurde sterreich

Topic #4:
mensdorff grad pouilly telekom philae


In [18]:
ldab.transform(XB)

array([[0.02892023, 0.02915217, 0.02895243, 0.88331973, 0.02965543],
       [0.02079293, 0.02070026, 0.02076849, 0.91702509, 0.02071323],
       [0.01625487, 0.01627112, 0.01628649, 0.93491749, 0.01627003],
       ...,
       [0.01455115, 0.01453421, 0.2423313 , 0.71397241, 0.01461093],
       [0.02118161, 0.02120742, 0.31930159, 0.61704532, 0.02126406],
       [0.02048033, 0.02043029, 0.22551352, 0.71311623, 0.02045963]])

# Title

In [19]:
df_2k['Title']=df_2k['Title'].apply(preprocessing) #applying preprocessing to Title Column

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
print(len(df_2k['Title'][0].split()))

2


In [21]:
print(len(df_2k['Title'][0]))

22


In [22]:
df_2k['Title'][0]

'newsletter derstandard'

In [23]:
print(len(df_2kt['Title'][0]))

33


In [24]:
tfidft = TfidfVectorizer() #defining tfidf vectorizer for title
c_corpus = ['{}'.format(text) for text in df_2k['Title']] #forming corpus
c_corpus
XT = tfidft.fit_transform(c_corpus) #fitting tfidf for title
f_names = tfidft.get_feature_names()
dense = XT.todense().tolist()
new_df = pd.DataFrame(dense,columns=f_names)
new_df.head()

,abbas,abbauarbeiten,abbauen,abbei,abberufung,abblitzen,abdullah,abflug,abgang,abgeblitzt,abgeordnete,abgeordneten,abgeordneter,abgesagt,abgewiesen,abgeworben,abgrund,ablenkungen,ablinger,abmahnungen,abregung,absagen,absatzschwund,abschied,abschlie,abschluss,abschlusspr,abschreckende,abschuss,absitzen,absolute,absolviert,abspielen,abstellgleis,abstieg,abstimmen,abstimmung,absturz,abwehrschlacht,abwertung,...,zittersieg,zivildieners,zivilisten,zivilklage,zocken,zotter,zuckerberg,zufrieden,zugang,zugeben,zugriff,zukunft,zukunftstr,zulieferern,zuordnen,zurechnungsf,zusammenarbeit,zusammenhang,zusammenprall,zusammensto,zuschauerzahl,zuseher,zustimmung,zuversichtlich,zuweisen,zwangsabstieg,zweckbindung,zwecks,zwei,zweier,zweifel,zweimal,zweite,zweiter,zweites,zwergplanet,zwischenf,zwischengelandet,zwist,zypriotischer
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
warnings.simplefilter("ignore", DeprecationWarning)
 
# Helper function
def print_topics(model, tfidft, n_top_words):
    words = tfidft.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below
number_topics = 5
number_words = 5
# Create and fit the LDA model
ldat = LDA(n_components=number_topics, n_jobs=-1)
ldat.fit(XT)
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(ldat, tfidft, number_words)

Topics found via LDA:

Topic #0:
windows macht verl juli euro

Topic #1:
sterreich forum wiener topic jahre

Topic #2:
wien zwei neuer salzburg trainer

Topic #3:
neue sterreich iran tote leben

Topic #4:
griechenland wien erste gestorben platz


In [26]:
ldat.transform(XT)

array([[0.08334152, 0.08347535, 0.086518  , 0.08312887, 0.66353626],
       [0.08321925, 0.0832779 , 0.66671315, 0.08312543, 0.08366428],
       [0.08401257, 0.08670577, 0.66249283, 0.08304162, 0.08374721],
       ...,
       [0.73156602, 0.06680943, 0.06680888, 0.06800592, 0.06680975],
       [0.17504178, 0.06201601, 0.63891133, 0.06201441, 0.06201647],
       [0.06755697, 0.7303677 , 0.06724237, 0.06733867, 0.06749428]])

# Cosine Similarity

In [0]:
body = df_2k['Body'] #making body df
title = df_2k['Title'] #making title df

In [0]:
blist = [] #making a list of unique words in body
bset = set()
for x in body:
    if x not in bset:
        blist.append(x)
        bset.add(x)

tlist = [] #making a list of unique words in title
tset = set()
for x in title:
    if x not in tset:
        tlist.append(x)
        tset.add(x)        

In [0]:
bvec = CountVectorizer().fit_transform(blist) #defining count vec for body list
vecb = bvec.toarray()

tvec = CountVectorizer().fit_transform(tlist) #defining count vec for title list
vect = tvec.toarray()

In [0]:
def cos_similarity(vec1,vec2): #defining function for cosine similarity
  vec1 = vec1.reshape(1,-1)
  vec2 = vec2.reshape(1,-1)

  return cosine_similarity(vec1,vec2)[0][0]

In [57]:
cos_similarity(vecb[0],vecb[1]) #finding cosine similarity for body 

0.30864159182697176

In [58]:
cos_similarity(vect[0],vect[1]) #finding cosine similarity for title

0.4999999999999999

# PyLDAVis

In [27]:
!pip install pyLDAvis #installing pyLDAvis

In [0]:
def tokenize(text): #function to tokenize text 
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in german_stop_words]
    return tokens

In [29]:
body_data = [] #creating empty list for body
for line in df_2k['Body']:
    tokens = tokenize(line)
    if random.random() > .99:
        print(tokens)
        body_data.append(tokens)

['zweite', 'runde', 'erraten', 'themen', 'ausgew', 'hlten', 'postings', 'ffentlicht', 'wurden', 'ersten', 'community', 'wissenschaft', 'zweite', 'runde', 'fachwissen', 'beitr', 'artikeln', 'vorgestellten', 'postings', 'finden', 'artikeln', 'ausschlie', 'wissenschaftsressort', 'ffentlicht', 'wurden', 'nschen', 'tseln', 'freuen', 'ckmeldung', 'forum', 'derstandard']
['spiel', 'mitte', 'hrlich', 'vorgestellt', 'square', 'ersten', 'teaser', 'trailer', 'raider', 'ffentlicht', 'junge', 'croft', 'ersten', 'abenteuer', 'zeigt', 'video', 'allerdings', 'sehen', 'croft', 'steigeisen', 'ausger', 'eisigen', 'erklimmt', 'gespielt', 'grabr', 'uberin', 'erneut', 'camilla', 'luddington', 'spiel', 'jahres', 'erscheinen', 'microsoft', 'konnte', 'zeitlich', 'exklusive', 'distribution', 'titels', 'sichern', 'raider', 'systemen', 'handel', 'kommen', 'erste', 'bilder', 'spiel', 'gaben', 'anfang', 'jahres', 'bereits', 'einblick', 'offizielle', 'vorstellung', 'branchemesse', 'stattfinden', 'thema', 'raider', '

In [30]:
dictb = corpora.Dictionary(body_data) #dictionary for body data
corpus_words_body = [dictb.doc2bow(text) for text in body_data]
pickle.dump(corpus_words_body, open('corpus.pkl', 'wb'))
dictb.save('dictionary_body.gensim')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [31]:
import gensim #finding topics for body
lda_model_body = gensim.models.ldamodel.LdaModel(corpus_words_body, num_topics = 5, id2word=dictb, passes=15)
lda_model_body.save('model_ldavis_b')
topicsb = lda_model_body.print_topics(num_words=4)
for topic in topicsb:
    print(topic)

(0, '0.008*"amazon" + 0.007*"jauch" + 0.006*"sendung" + 0.005*"peking"')
(1, '0.008*"prozent" + 0.007*"jahren" + 0.007*"pesos" + 0.007*"aserbaidschan"')
(2, '0.011*"menschen" + 0.010*"wajcman" + 0.009*"bhutan" + 0.007*"standard"')
(3, '0.008*"tochter" + 0.007*"bisher" + 0.007*"bayern" + 0.005*"millionen"')
(4, '0.008*"gazprom" + 0.008*"turkstream" + 0.006*"wenig" + 0.006*"realisierung"')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [32]:
dictionaryb = gensim.corpora.Dictionary.load('dictionary_body.gensim') #display of the pyLDAvis for body
corpus_b = pickle.load(open('corpus.pkl', 'rb'))
ldab = gensim.models.ldamodel.LdaModel.load('model_ldavis_b')
import pyLDAvis.gensim
lda_displayb = pyLDAvis.gensim.prepare(ldab, corpus_b, dictionaryb, sort_topics=False)
pyLDAvis.display(lda_displayb)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [33]:
title_data = [] #creating empty list for title
for line in df_2k['Title']:
    tokens = tokenize(line)
    if random.random() > .99:
        print(tokens)
        title_data.append(tokens)

['ltnis', 'tschechien', 'russland', 'belastet']
['morrison', 'spotlight', 'schamanen']
['polizist', 'quittierte', 'brutalem', 'einsatz', 'poolparty', 'dienst']
['waffenproduzent', 'pleite']
['nationalbibliothek', 'parlamentarier', 'aufnehmen']
['jauch', 'drohte', 'schon', 'ndigung']
['hitzeproblem', 'xperia']
['japaner', 'schlafen', 'wenig']
['polizei', 'ermittelt', 'wegen', 'rassistischen', 'manifests']
['rauscher', 'robert', 'misik', 'griechenland', 'krise']
['amazon', 'streik', 'weitet']
['derstandard', 'freut', 'facebook']
['geisterfahrer', 'internet']
['batman', 'arkham', 'knight', 'hersteller', 'wusste', 'offenbar', 'monaten', 'problemen']
['kupferpreis', 'bricht', 'wegen', 'angst', 'china', 'krise']
['verhandlungsmarathon', 'grexit', 'tisch', 'vorerst']
['startet', 'personalisierten', 'nachrichten']
['nachfrage', 'wiener', 'romarkt']
['syrien', 'sident', 'assad', 'deserteuren', 'amnestie']


In [34]:
dictt = corpora.Dictionary(title_data) #dictionary for title data
corpus_words_title = [dictt.doc2bow(text) for text in title_data]

pickle.dump(corpus_words_title, open('corpus.pkl', 'wb'))
dictt.save('dictionary_title.gensim')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [35]:
import gensim #finding topics for title
lda_model_title = gensim.models.ldamodel.LdaModel(corpus_words_title, num_topics = 5, id2word=dictt, passes=15)
lda_model_title.save('model_ldavis_t')
topicst = lda_model_title.print_topics(num_words=4)
for topic in topicst:
    print(topic)

(0, '0.045*"wegen" + 0.024*"poolparty" + 0.024*"einsatz" + 0.024*"quittierte"')
(1, '0.051*"japaner" + 0.051*"wenig" + 0.051*"schlafen" + 0.051*"internet"')
(2, '0.047*"assad" + 0.047*"amnestie" + 0.047*"sident" + 0.047*"deserteuren"')
(3, '0.036*"krise" + 0.036*"batman" + 0.036*"arkham" + 0.036*"wusste"')
(4, '0.014*"morrison" + 0.014*"schamanen" + 0.014*"wiener" + 0.014*"nachfrage"')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [36]:
dictionaryt = gensim.corpora.Dictionary.load('dictionary_title.gensim') #display of the pyLDAvis for body
corpus_t = pickle.load(open('corpus.pkl', 'rb'))
ldat = gensim.models.ldamodel.LdaModel.load('model_ldavis_t')
import pyLDAvis.gensim
lda_displayt = pyLDAvis.gensim.prepare(ldat, corpus_t, dictionaryt, sort_topics=False)
pyLDAvis.display(lda_displayt)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
